<a href="https://colab.research.google.com/github/ProfAI/machine-learning-modelli-e-algoritmi/blob/main/4%20-%20SVM/svm_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previsione di infarto

Per questa esercitazione dovrai utilizzare un dataset contenente diverse informazioni ed esami a pazienti per prevedere chi di loro è a rischio infarto. Quale metrica andrebbe ottimizzata in questo caso? Realizza un primo modello baseline e poi uno utilizzando le SVM.

Puoi scaricare il dataset [da qui](https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset), di seguito le variabili che contiene:

 - **age**: Età del paziente

 - **sex**: Sesso del paziente

 - **cp**: Tipo di dolore toracico, 0 = Angina tipica, 1 = Angina atipica, 2 = Dolore non anginoso, 3 = Asintomatico

 - **trtbps**: Pressione arteriosa a riposo (in mm Hg)

 - **col**: Colesterolo in mg/dl rilevato tramite il sensore BMI

 - **fbs**: (glicemia a digiuno > 120 mg/dl), 1 = vero, 0 = falso

 - **restecg**: Risultati elettrocardiografici a riposo, 0 = normale, 1 = normalità dell'onda ST-T, 2 = ipertrofia ventricolare sinistra

 - **thalachh**: Frequenza cardiaca massima raggiunta

 - **oldpeak**: Picco precedente

 - **slp**: Pendenza

 - **caa**: Numero di vasi principali

 - **thall**: Risultato dello Stress Test al Tallio ~ (0,3)

 - **exng**: Angina indotta dall'esercizio ~ 1 = Sì, 0 = No

 - **output**: Variabile target

### Importiamo i moduli

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.compose import ColumnTransformer

### Definiamo le costanti

In [2]:
BASE_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-modelli-e-algoritmi/main/datasets/"
RANDOM_SEED = 0

### Definiamo le funzioni

In [3]:
def classifier_report(model, data):
  X, y = data
  y_pred = model.predict(X)
  report = classification_report(y, y_pred)
  print(report)
  print(confusion_matrix(y, y_pred))

### Importiamo il dataset


In [4]:
df = pd.read_csv(BASE_URL+"heart.csv")
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [6]:
df.describe()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [7]:
#sns.pairplot(df)
#plt.show()

## Processiamo i dati

In [8]:
df.drop_duplicates(keep='first',inplace=True)

In [9]:
df.describe()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,302.00000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000
mean,54.42053,0.682119,0.963576,131.602649,246.500000,0.149007,0.526490,149.569536,0.327815,1.043046,1.397351,0.718543,2.314570,0.543046
std,9.04797,0.466426,1.032044,17.563394,51.753489,0.356686,0.526027,22.903527,0.470196,1.161452,0.616274,1.006748,0.613026,0.498970
min,29.00000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48.00000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.250000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.50000,1.000000,1.000000,130.000000,240.500000,0.000000,1.000000,152.500000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.00000,1.000000,2.000000,140.000000,274.750000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.00000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [10]:
X = df.drop(["output"], axis=1).values
y = df["output"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=RANDOM_SEED)

In [11]:
# One hot encoding

ct = ColumnTransformer(
    [
        ("ohe", OneHotEncoder(), [2]),  #il terzo parametro indica la terza colonna cp
    ],
    remainder = "passthrough"  # le colonne non trasformate devono rimanere
)

In [12]:
# Standardizziamo
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [13]:
X_train.shape

(241, 13)

### Modello baseline - Regressione Lineare

In [14]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.8688524590163934

In [15]:
classifier_report(lr, (X_test, y_test))

              precision    recall  f1-score   support

           0       0.88      0.81      0.85        27
           1       0.86      0.91      0.89        34

    accuracy                           0.87        61
   macro avg       0.87      0.86      0.87        61
weighted avg       0.87      0.87      0.87        61

[[22  5]
 [ 3 31]]


Qual è la metrica che dobbiamo considerare maggiormente ? 
Se un paziente che effettivamente non è a rischio infarto viene classificato come a rischio infarto allora su di lui verranno fatti altri esami e si scoprirà che il modello aveva sbaglaito, ma non sarà un grosso problema. (Questa cosa viene indicata dalla Precision )

Al contrario se un paziente che effettivamente è a rischio infarto ma viene classificato come non a rischio allora la situazione è più grave. (Questa cosa viene indicata dalla Recall)

La Recall al momento è abbastanza alta. 

I false negative in questo caso sono 3, che è il valore che ci interessa di più in questo caso.
Riusciamo a ridurre questo valore a 3 senza peggiorare le altre metriche ?

### SVM Lineare

In [16]:
svc = SVC(kernel="linear")
svc.fit(X_train, y_train)
svc.score(X_test, y_test)

0.8524590163934426

In [17]:
classifier_report(svc, (X_test, y_test))

              precision    recall  f1-score   support

           0       0.85      0.81      0.83        27
           1       0.86      0.88      0.87        34

    accuracy                           0.85        61
   macro avg       0.85      0.85      0.85        61
weighted avg       0.85      0.85      0.85        61

[[22  5]
 [ 4 30]]


### Kernel SVM

In [18]:
svc = SVC(kernel="rbf")
svc.fit(X_train, y_train)
svc.score(X_test, y_test)

0.9344262295081968

In [19]:
classifier_report(svc, (X_test, y_test))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93        27
           1       0.97      0.91      0.94        34

    accuracy                           0.93        61
   macro avg       0.93      0.94      0.93        61
weighted avg       0.94      0.93      0.93        61

[[26  1]
 [ 3 31]]


### Kernel SVM con classi pesate

In [25]:
svc = SVC(kernel="rbf", class_weight={1:1.5})
svc.fit(X_train, y_train)
svc.score(X_test, y_test)

0.9016393442622951

In [26]:
classifier_report(svc, (X_test, y_test))

              precision    recall  f1-score   support

           0       0.92      0.85      0.88        27
           1       0.89      0.94      0.91        34

    accuracy                           0.90        61
   macro avg       0.90      0.90      0.90        61
weighted avg       0.90      0.90      0.90        61

[[23  4]
 [ 2 32]]


Soluzione Migliore anche se rimangono ancora 2 falsi negativi

In [22]:
svc.n_support_

array([77, 66], dtype=int32)

### Cross Validation